In [1]:
from src.datasets.data_module import CloudCoverDataModule
from pathlib import Path
from src.models.unet import LightningUNet
from src.training.trainer import train
from src.testing.tester import test

### Dataloader

In [2]:
data_module = CloudCoverDataModule(
    train_X_folder_path=Path("../data/final/public/train_features/"),
    train_y_folder_path=Path("../data/final/public/train_labels/"),
    test_X_folder_path=Path("../data/final/private/test_features/"),
    test_y_folder_path=Path("../data/final/private/test_labels/"),
    train_batch_size=4,
    val_batch_size=8,
    test_batch_size=8,
    val_size=0.2,
    random_state=42
)

In [3]:
data_module.prepare_data()

In [4]:
data_module.setup(stage="fit")
data_module.setup(stage="test")

### Train

In [5]:
unet = train(
    model=LightningUNet(n_channels=4, n_classes=2, bilinear=True, learning_rate=0.001),
    run_name="unet",
    model_version=0,
    data_module=data_module,
    max_epochs=35,
    patience=5
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type               | Params
-------------------------------------------------------
0  | train_jaccard  | BinaryJaccardIndex | 0     
1  | train_accuracy | BinaryAccuracy     | 0     
2  | val_jaccard    | BinaryJaccardIndex | 0     
3  | val_accuracy   | BinaryAccuracy     | 0     
4  | test_jaccard   | BinaryJaccardIndex | 0     
5  | test_accuracy  | BinaryAccuracy     | 0     
6  | inc            | DoubleConv         | 39.6 K
7  | down1          | Down               | 221 K 
8  | down2          | Down               | 886 K 
9  | down3          | Down               | 3.5 M 
10 | down4          | Down               | 4.7 M 
11 | up1            | Up                 | 5.9 M 
12 | up2            | Up                 | 1.5 M 
13 | up3            | Up                 | 369 K 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/user/micromamba/envs/gif-7005-project/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/user/micromamba/envs/gif-7005-project/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


/home/user/micromamba/envs/gif-7005-project/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 1:   3%|▎         | 76/2350 [00:44<22:05,  1.72it/s, v_num=1, train_loss_step=0.113, train_jaccard_step=0.937, train_accuracy_step=0.964, val_loss_step=0.510, val_jaccard_step=0.628, val_accuracy_step=0.786, val_loss_epoch=0.370, val_jaccard_epoch=0.791, val_accuracy_epoch=0.849, train_loss_epoch=0.380, train_jaccard_epoch=0.751, train_accuracy_epoch=0.840]  

/home/user/micromamba/envs/gif-7005-project/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


### Test  
Replace the checkpoint path with the best checkpoint from the training.

In [5]:
unet = LightningUNet.load_from_checkpoint(checkpoint_path='./logs/unet/version_0/checkpoints/unet-epoch=07-val_loss=0.26.ckpt', n_channels=4, n_classes=2)

In [6]:
test(
    model=unet,
    run_name="unet",
    model_version=0,
    data_module=data_module
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/user/micromamba/envs/gif-7005-project/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0:   0%|          | 0/1373 [00:00<?, ?it/s]

/home/user/micromamba/envs/gif-7005-project/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Testing DataLoader 0: 100%|██████████| 1373/1373 [10:09<00:00,  2.25it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_jaccard_epoch       0.8518288135528564
     test_loss_epoch        0.27107203006744385
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
